In [47]:
# Test inverse transform of pystackreg registration.
# Question: is there a simple inverse transformation matrix for affine and bilinear transformation?
# I could inverse registration for rigid body by applying inverse matrix of the transformation matrix, but could not do that with affine or bilinear.
# Test: From real session-to-session registration examples, perform registration in both ways.
# Compare the matrices. 
# Find rules if possible.

import numpy as np
from pystackreg import StackReg
import napari


h5Dir = 'H:/'
mice =          [25,    27,   30,   36,     39,     52]
refSessions =   [4,     3,    3,    1,      1,      3]
expSessions =   [19,    10,   21,   17,     23,     21]
zoom =          [2,     2,    2,    1.7,    1.7,    1.7]
freq =          [7.7,   7.7,  7.7,  7.7,    6.1,    7.7]

optimal_reg_methods = {'025': ['bilinear', 'bilinear', 'suite2p', 'bilinear', 'old', 'old', 'old', 'old'],
                        '027': ['bilinear', 'bilinear', 'bilinear', 'bilinear', 'suite2p', 'suite2p', 'affine', 'bilinear'],
                        '030': ['old', 'old', 'old', 'bilinear', 'affine', 'bilinear', 'bilinear', 'bilinear'],
                        '036': ['old', 'old', 'old', 'suite2p', 'suite2p', 'bilinear', 'old', 'bilinear'],
                        '039': ['suite2p', 'suite2p', 'suite2p', 'suite2p', 'affine', 'affine', 'affine', 'affine'],
                        '052': ['bilinear', 'bilinear', 'bilinear', 'bilinear', 'bilinear', 'bilinear', 'bilinear', 'affine']}
relative_depths_selected = {'025': [[7,17], [18,28]],
                            '027': [[20,30], [25,35]],
                            '030': [[17,27], [22,32]],
                            '036': [[16,26], [12,22]],
                            '039': [[22,32], [17,27]],
                            '052': [[27,37], [17,27]]} # per imaged volume
manual_removal_Session_i = {'025': [[7,14], []],
                            '027': [range(7,12), []],
                            '030': [[], [19]],
                            '036': [[17], [19]],
                            '039': [[], []],
                            '052': [[], []]} # per imaged volume

def imblend_for_napari(refImg, testImg):
    if (len(refImg.shape) != 2) or (len(testImg.shape) != 2):
        raise('Both images should have 2 dims.')
    if any(np.array(refImg.shape)-np.array(testImg.shape)):
        raise('Both images should have matching dims')
    refImg = img_norm(refImg.copy())
    testImg = img_norm(testImg.copy())
    refRGB = np.moveaxis(np.tile(refImg,(3,1,1)), 0, -1)
    testRGB = np.moveaxis(np.tile(testImg,(3,1,1)), 0, -1)
    blended = imblend(refImg, testImg)
    return np.array([refRGB, testRGB, blended])

def img_norm(img):
    # return (img - np.amin(img)) / (np.amax(img) - np.amin(img))
    img = np.clip(img, np.percentile(img,0.2), np.percentile(img, 99.8))
    return (img - np.amin(img)) / (np.amax(img) - np.amin(img))

def imblend(refImg, testImg):
    if (len(refImg.shape) != 2) or (len(testImg.shape) != 2):
        raise('Both images should have 2 dims.')
    if any(np.array(refImg.shape)-np.array(testImg.shape)):
        raise('Both images should have matching dims')
    Ly,Lx = refImg.shape
    blended = np.zeros((Ly,Lx,3))
    blended[:,:,0] = refImg
    blended[:,:,2] = testImg
    blended[:,:,1] = refImg
    return blended

In [6]:
mi = 0
pn = 2
mouse=mice[mi]
mouse_str = f'{mouse:03}'
reg_meth = optimal_reg_methods[mouse_str][pn-1]
refSn = refSessions[mi] # for 'old' method
planeDir = f'{h5Dir}{mouse:03}/plane_{pn}/'
session_to_session_registration = np.load(f'{planeDir}JK{mouse:03}_plane{pn}_session_to_session_registration.npy', allow_pickle=True).item()
#  result = {'mouse': mouse,
#                 'plane': pn,
#                 'edge_buffer': edge_buffer,
#                 'selected_session_i': selectedSi,
#                 'selected_session_num': selectedSnums,
#                 'zdrift_list': zdrift_list,
#                 'registration_method': reg_meth,
#                 'bilinear_result': bilinear_result, 
#                 'affine_result': affine_result,
#                 'suite2p_result': suite2p_result,
#                 'old_result': old_result
#                 }
selected_session_num = session_to_session_registration['selected_session_num']
edge_buffer = session_to_session_registration['edge_buffer']
reg_meth

'bilinear'

In [5]:
selected_session_num

array([ 1,  2,  3,  4,  5,  6, 13, 15, 17, 19, 22, 23, 24])

In [12]:
def get_mimg(planeDir,session_num,edge_buffer):
    opsFn = f'{planeDir}{session_num:03}/plane0/ops.npy'
    ops = np.load(opsFn, allow_pickle=True).item()
    topBuffer = edge_buffer['topBuffer']
    bottomBuffer = edge_buffer['bottomBuffer']
    leftBuffer = edge_buffer['leftBuffer']
    rightBuffer = edge_buffer['rightBuffer']
    mimg = ops['meanImg'][topBuffer:-bottomBuffer,leftBuffer:-rightBuffer]
    return mimg
ref_i = 0
mov_i = 10
ref_num = selected_session_num[ref_i]
mov_num = selected_session_num[mov_i]
ref_mimg = get_mimg(planeDir, ref_num, edge_buffer)
mov_mimg = get_mimg(planeDir, mov_num, edge_buffer)
napari.view_image(np.array([ref_mimg,mov_mimg]))

c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 163.0, 313.0), zoom=2.4736421725239617, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(0.0, 0.0, 0.0), scaled=True, size=1, style='standard'), dims=Dims(ndim=3, ndisplay=2, last_used=2, range=((0.0, 1.0, 1.0), (0.0, 326.0, 1.0), (0.0, 626.0, 1.0)), current_step=(0, 0, 0), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(enabled=False, stride=1, shape=(-1, -1)), layers=[<Image layer 'Image' at 0x1758496e880>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position='bottom_right', font_size=10.0, unit=None), text_overlay=TextOverlay(visible=False, color=array([1., 1., 1., 1.]), font_size=10.0, position='top_left', text=''), help='', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_wheel_callbacks=[<function dims_scrol

In [20]:
# Rigid transformation bidirectionally
sr = StackReg(StackReg.RIGID_BODY)
tmat_mov_to_ref = sr.register(ref_mimg,mov_mimg)
reg_mimg_mov_to_ref = sr.transform(mov_mimg, tmat=tmat_mov_to_ref)
tmat_mov_to_ref

array([[ 0.99966222,  0.02598926,  2.97495359],
       [-0.02598926,  0.99966222, 10.48029864],
       [ 0.        ,  0.        ,  1.        ]])

In [16]:
sr = StackReg(StackReg.RIGID_BODY)
tmat_ref_to_mov = sr.register(mov_mimg, ref_mimg)
reg_mimg_ref_to_mov = sr.transform(ref_mimg, tmat=tmat_ref_to_mov)
tmat_ref_to_mov

array([[  0.99966286,  -0.02596487,  -2.63573547],
       [  0.02596487,   0.99966286, -10.50004868],
       [  0.        ,   0.        ,   1.        ]])

In [17]:
napari.view_image(np.array([reg_mimg_ref_to_mov, mov_mimg]))


c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 163.0, 313.0), zoom=2.4736421725239617, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(0.0, 0.0, 0.0), scaled=True, size=1, style='standard'), dims=Dims(ndim=3, ndisplay=2, last_used=2, range=((0.0, 1.0, 1.0), (0.0, 326.0, 1.0), (0.0, 626.0, 1.0)), current_step=(0, 0, 0), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(enabled=False, stride=1, shape=(-1, -1)), layers=[<Image layer 'Image' at 0x1758590f2e0>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position='bottom_right', font_size=10.0, unit=None), text_overlay=TextOverlay(visible=False, color=array([1., 1., 1., 1.]), font_size=10.0, position='top_left', text=''), help='', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_wheel_callbacks=[<function dims_scrol

In [18]:
napari.view_image(np.array([reg_mimg_mov_to_ref, ref_mimg]))

c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 163.0, 313.0), zoom=2.4736421725239617, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(0.0, 0.0, 0.0), scaled=True, size=1, style='standard'), dims=Dims(ndim=3, ndisplay=2, last_used=2, range=((0.0, 1.0, 1.0), (0.0, 326.0, 1.0), (0.0, 626.0, 1.0)), current_step=(0, 0, 0), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(enabled=False, stride=1, shape=(-1, -1)), layers=[<Image layer 'Image' at 0x17588105d00>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position='bottom_right', font_size=10.0, unit=None), text_overlay=TextOverlay(visible=False, color=array([1., 1., 1., 1.]), font_size=10.0, position='top_left', text=''), help='', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_wheel_callbacks=[<function dims_scrol

In [21]:
np.matmul(tmat_mov_to_ref, tmat_ref_to_mov)

array([[ 1.00000000e+00,  2.43963666e-05,  6.72199545e-02],
       [-2.43963666e-05,  1.00000000e+00,  5.22974502e-02],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [22]:
# Apply two transformations and see if it comes back to the original image
sr = StackReg(StackReg.RIGID_BODY)
ref_comeback_mimg = sr.transform(sr.transform(ref_mimg, tmat=tmat_ref_to_mov), tmat=tmat_mov_to_ref)
napari.view_image(np.array([ref_mimg, ref_comeback_mimg]))


c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 163.0, 313.0), zoom=2.4736421725239617, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(0.0, 0.0, 0.0), scaled=True, size=1, style='standard'), dims=Dims(ndim=3, ndisplay=2, last_used=2, range=((0.0, 1.0, 1.0), (0.0, 326.0, 1.0), (0.0, 626.0, 1.0)), current_step=(0, 0, 0), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(enabled=False, stride=1, shape=(-1, -1)), layers=[<Image layer 'Image' at 0x1758913dd90>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position='bottom_right', font_size=10.0, unit=None), text_overlay=TextOverlay(visible=False, color=array([1., 1., 1., 1.]), font_size=10.0, position='top_left', text=''), help='', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_wheel_callbacks=[<function dims_scrol

In [23]:
sr = StackReg(StackReg.RIGID_BODY)
mov_comeback_mimg = sr.transform(sr.transform(mov_mimg, tmat=tmat_mov_to_ref), tmat=tmat_ref_to_mov)
napari.view_image(np.array([mov_mimg, mov_comeback_mimg]))

c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 163.0, 313.0), zoom=2.4736421725239617, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(0.0, 0.0, 0.0), scaled=True, size=1, style='standard'), dims=Dims(ndim=3, ndisplay=2, last_used=2, range=((0.0, 1.0, 1.0), (0.0, 326.0, 1.0), (0.0, 626.0, 1.0)), current_step=(0, 0, 0), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(enabled=False, stride=1, shape=(-1, -1)), layers=[<Image layer 'Image' at 0x1758a477400>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position='bottom_right', font_size=10.0, unit=None), text_overlay=TextOverlay(visible=False, color=array([1., 1., 1., 1.]), font_size=10.0, position='top_left', text=''), help='', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_wheel_callbacks=[<function dims_scrol

In [24]:
# comeback image from applying inverse matrix
inv_tmat_ref_to_mov = np.linalg.inv(tmat_ref_to_mov)
ref_comeback_mimg_using_inv = sr.transform(sr.transform(ref_mimg, tmat=tmat_ref_to_mov), inv_tmat_ref_to_mov)
napari.view_image(np.array([ref_mimg, ref_comeback_mimg_using_inv]))

c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 163.0, 313.0), zoom=2.4736421725239617, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(0.0, 0.0, 0.0), scaled=True, size=1, style='standard'), dims=Dims(ndim=3, ndisplay=2, last_used=2, range=((0.0, 1.0, 1.0), (0.0, 326.0, 1.0), (0.0, 626.0, 1.0)), current_step=(0, 0, 0), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(enabled=False, stride=1, shape=(-1, -1)), layers=[<Image layer 'Image' at 0x1758b7c6c70>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position='bottom_right', font_size=10.0, unit=None), text_overlay=TextOverlay(visible=False, color=array([1., 1., 1., 1.]), font_size=10.0, position='top_left', text=''), help='', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_wheel_callbacks=[<function dims_scrol

# Affine

In [25]:
# Now, do the same thing with affine transformation
sr = StackReg(StackReg.AFFINE)
tmat_mov_to_ref = sr.register(ref_mimg, mov_mimg)
reg_mimg_mov_to_ref = sr.transform(mov_mimg, tmat=tmat_mov_to_ref)
tmat_mov_to_ref

array([[ 0.98454682,  0.02226894,  6.75687113],
       [-0.02575935,  0.97789764, 13.98751523],
       [ 0.        ,  0.        ,  1.        ]])

In [26]:
tmat_ref_to_mov = sr.register(mov_mimg, ref_mimg)
reg_mimg_ref_to_mov = sr.transform(ref_mimg, tmat=tmat_ref_to_mov)
tmat_ref_to_mov

array([[  1.01726951,  -0.02682832,  -6.36392689],
       [  0.02523787,   1.03109363, -15.5227741 ],
       [  0.        ,   0.        ,   1.        ]])

In [27]:
np.matmul(tmat_ref_to_mov,tmat_mov_to_ref)
# Affine transformation back and forth does not have inverse matrix relationship

array([[ 1.00224054, -0.00358183,  0.13437055],
       [-0.00171244,  1.00886606, -0.92980718],
       [ 0.        ,  0.        ,  1.        ]])

In [32]:
napari.view_image(np.array([reg_mimg_ref_to_mov, mov_mimg]));

c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


In [30]:
napari.view_image(np.array([reg_mimg_mov_to_ref, ref_mimg]));

c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 163.0, 313.0), zoom=2.4736421725239617, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(0.0, 0.0, 0.0), scaled=True, size=1, style='standard'), dims=Dims(ndim=3, ndisplay=2, last_used=2, range=((0.0, 1.0, 1.0), (0.0, 326.0, 1.0), (0.0, 626.0, 1.0)), current_step=(0, 0, 0), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(enabled=False, stride=1, shape=(-1, -1)), layers=[<Image layer 'Image' at 0x1758ff6d070>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position='bottom_right', font_size=10.0, unit=None), text_overlay=TextOverlay(visible=False, color=array([1., 1., 1., 1.]), font_size=10.0, position='top_left', text=''), help='', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_wheel_callbacks=[<function dims_scrol

In [31]:
mov_comeback_mimg = sr.transform(sr.transform(mov_mimg, tmat=tmat_mov_to_ref), tmat=tmat_ref_to_mov)
napari.view_image(np.array([mov_mimg, mov_comeback_mimg]))
# Applying back and forth does not make it come back to the original image.

c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 163.0, 313.0), zoom=2.4736421725239617, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(0.0, 0.0, 0.0), scaled=True, size=1, style='standard'), dims=Dims(ndim=3, ndisplay=2, last_used=2, range=((0.0, 1.0, 1.0), (0.0, 326.0, 1.0), (0.0, 626.0, 1.0)), current_step=(0, 0, 0), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(enabled=False, stride=1, shape=(-1, -1)), layers=[<Image layer 'Image' at 0x175929ba5b0>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position='bottom_right', font_size=10.0, unit=None), text_overlay=TextOverlay(visible=False, color=array([1., 1., 1., 1.]), font_size=10.0, position='top_left', text=''), help='', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_wheel_callbacks=[<function dims_scrol

In [34]:
# How about registering the registered image? (round-trip)
tmat_ref_round_trip = sr.register(ref_mimg, reg_mimg_ref_to_mov)
ref_mimg_round_trip = sr.transform(sr.transform(ref_mimg, tmat=tmat_ref_to_mov), tmat=tmat_ref_round_trip)
napari.view_image(np.array([ref_mimg, ref_mimg_round_trip]));
# Round-trip restores the image perfectly.

c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


In [35]:
np.matmul(tmat_ref_to_mov,tmat_ref_round_trip)
# In case of affine, it is very close to the inverse transformation.

array([[9.99992760e-01, 9.95226745e-06, 1.33449972e-03],
       [7.75027883e-06, 9.99955193e-01, 7.27147807e-03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [37]:
# Try inverse transformation for affine
inv_tmat_ref_to_mov = np.linalg.inv(tmat_ref_to_mov)
ref_comeback_mimg_using_inv = sr.transform(sr.transform(ref_mimg, tmat=tmat_ref_to_mov), inv_tmat_ref_to_mov)
napari.view_image(np.array([ref_mimg, ref_comeback_mimg_using_inv]));
# For affine, inverse matrix works.

c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


# Bilinear

In [39]:
# Now, test in bilinear transformation
sr = StackReg(StackReg.BILINEAR)
tmat_mov_to_ref = sr.register(ref_mimg, mov_mimg)
reg_mimg_mov_to_ref = sr.transform(mov_mimg, tmat=tmat_mov_to_ref)
tmat_mov_to_ref

array([[ 9.82053818e-01,  1.84835147e-02,  1.33689915e-05,
         7.43799346e+00],
       [-2.54524518e-02,  9.78127100e-01, -1.26218533e-06,
         1.39315565e+01],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [40]:
tmat_ref_to_mov = sr.register(mov_mimg, ref_mimg)
reg_mimg_ref_to_mov = sr.transform(ref_mimg, tmat=tmat_ref_to_mov)
tmat_ref_to_mov

array([[ 1.02058613e+00, -1.53341001e-02, -2.69962849e-05,
        -8.02879803e+00],
       [ 2.75040379e-02,  1.02549153e+00, -5.31253373e-06,
        -1.49557850e+01],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [41]:
np.matmul(tmat_ref_to_mov,tmat_mov_to_ref)
# Bilinear transformation back and forth does not have inverse matrix relationship

array([[ 1.00266079e+00,  3.86531977e-03, -1.33327232e-05,
        -6.51312983e-01],
       [ 9.09171742e-04,  1.00356943e+00, -6.23919284e-06,
        -4.64517010e-01],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [42]:
napari.view_image(np.array([reg_mimg_ref_to_mov, mov_mimg]));

c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


In [43]:
napari.view_image(np.array([reg_mimg_mov_to_ref, ref_mimg]));

c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


In [51]:
mov_comeback_mimg = sr.transform(sr.transform(mov_mimg, tmat=tmat_mov_to_ref), tmat=tmat_ref_to_mov)
# napari.view_image(imblend_for_napari(mov_mimg, mov_comeback_mimg));
viewer = napari.Viewer()
viewer.add_image(mov_comeback_mimg)
viewer.add_image(mov_mimg)
# Applying back and forth is close to the original image - but maybe it is specific to this image pair.



c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


<Image layer 'mov_mimg' at 0x1759c575310>

# For bilinear registration
- Are samples registered with bilinear transformation works well with back and forth?
- If not, I should apply round=trip registration method.

In [54]:
mi = 0
pn = 4
mouse=mice[mi]
mouse_str = f'{mouse:03}'
reg_meth = optimal_reg_methods[mouse_str][pn-1]
refSn = refSessions[mi] # for 'old' method
planeDir = f'{h5Dir}{mouse:03}/plane_{pn}/'
session_to_session_registration = np.load(f'{planeDir}JK{mouse:03}_plane{pn}_session_to_session_registration.npy', allow_pickle=True).item()
selected_session_num = session_to_session_registration['selected_session_num']
edge_buffer = session_to_session_registration['edge_buffer']
reg_meth

'bilinear'

In [59]:
ref_i = 3
mov_i = 8
ref_num = selected_session_num[ref_i]
mov_num = selected_session_num[mov_i]
ref_mimg = get_mimg(planeDir, ref_num, edge_buffer)
mov_mimg = get_mimg(planeDir, mov_num, edge_buffer)
napari.view_image(np.array([ref_mimg,mov_mimg]))

c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 163.0, 313.0), zoom=2.4736421725239617, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(0.0, 0.0, 0.0), scaled=True, size=1, style='standard'), dims=Dims(ndim=3, ndisplay=2, last_used=2, range=((0.0, 1.0, 1.0), (0.0, 326.0, 1.0), (0.0, 626.0, 1.0)), current_step=(0, 0, 0), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(enabled=False, stride=1, shape=(-1, -1)), layers=[<Image layer 'Image' at 0x17593a4f9d0>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position='bottom_right', font_size=10.0, unit=None), text_overlay=TextOverlay(visible=False, color=array([1., 1., 1., 1.]), font_size=10.0, position='top_left', text=''), help='', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_wheel_callbacks=[<function dims_scrol

In [61]:
sr = StackReg(StackReg.BILINEAR)
tmat_mov_to_ref = sr.register(ref_mimg, mov_mimg)
reg_mimg_mov_to_ref = sr.transform(mov_mimg, tmat=tmat_mov_to_ref)
tmat_ref_to_mov = sr.register(mov_mimg, ref_mimg)
reg_mimg_ref_to_mov = sr.transform(ref_mimg, tmat=tmat_ref_to_mov)
viewer = napari.Viewer()
viewer.add_image(np.array([reg_mimg_ref_to_mov, mov_mimg]), name='ref to mov');
viewer.add_image(np.array([reg_mimg_mov_to_ref, ref_mimg]), name='mov to ref');

c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


Examples of bad registration <br>
mi = 0, pn = 4, ref_i = 3, mov_i = 8: ref to mov is great, mov to ref is bad -> both mov comeback and ref comeback were bad.
- Cannot use comeback registration

In [62]:
mov_comeback_mimg = sr.transform(sr.transform(mov_mimg, tmat=tmat_mov_to_ref), tmat=tmat_ref_to_mov)
viewer = napari.Viewer()
viewer.add_image(mov_comeback_mimg)
viewer.add_image(mov_mimg)

c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


<Image layer 'mov_mimg' at 0x17592a6e370>

In [63]:
ref_comeback_mimg = sr.transform(sr.transform(ref_mimg, tmat=tmat_ref_to_mov), tmat=tmat_mov_to_ref)
viewer = napari.Viewer()
viewer.add_image(ref_comeback_mimg)
viewer.add_image(ref_mimg)

c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


<Image layer 'ref_mimg' at 0x17599620910>

What about round-trip registration in this case?

In [65]:
# How about registering the registered image? (round-trip)
tmat_ref_round_trip = sr.register(ref_mimg, reg_mimg_ref_to_mov)
ref_mimg_round_trip = sr.transform(sr.transform(ref_mimg, tmat=tmat_ref_to_mov), tmat=tmat_ref_round_trip)
tmat_mov_round_trip = sr.register(mov_mimg, reg_mimg_mov_to_ref)
mov_mimg_round_trip = sr.transform(sr.transform(mov_mimg, tmat=tmat_mov_to_ref), tmat=tmat_mov_round_trip)

viewer = napari.Viewer()
viewer.add_image(np.array([ref_mimg, ref_mimg_round_trip]), name='ref round-trip');
viewer.add_image(np.array([mov_mimg, mov_mimg_round_trip]), name='mov round-trip');


c:\Users\shires\Anaconda3\envs\suite2p\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


For bad registration (mov-to-ref in this example), round-trip was better than back and forth, but still it was quite distorted. <br>
For good registraiton (ref-to-mov in this example), round-trip perfectly restored the image.

# Conclusion
- For affine, apply inverse matrix for transformation.
- For bilinear, since I confirmed it worked for select samples, apply the round-trip registration for each session image.

# Next step
- Visually check how the backward registration (from the master map to each session) worked.
- Quantify proportion of matched area.
    - Can be used to filter out ROIs to discard. (Maybe apply to a certain region of the FOV?) 
        - May not need to discard ROIs at all. 
        - Because I want to look at non-detected ROIs as well. 
        - Also, suite2p does not get all active cells, and I curated very conservatively.
    - Possibly in combination with the signal correlation values.
- Collect signals in both this backward registration (With lambda=1) and the session ROI (With lambda=1)
    - Quantify correlation between these two signals. 